# Tutorial on Einstein's Summation Convention by using Numpy

In [1]:
import matplotlib.pyplot as plt
import imageio
import os
import sys
import skimage as ski
import time
import numpy as np

print(os.getcwd())
sys.path.append(os.path.abspath("../src/"))
from inverse_compositional_algorithm import inverse_compositional_algorithm, robust_inverse_compositional_algorithm, pyramidal_inverse_compositional_algorithm

/mnt/git/computational_astro/inverse_compositional_algorithm/test


## Some examples of Einstein's Summation Convention

### 1. Matrix transpose

In [2]:
a = np.arange(6).reshape(2,3)

print("a = ")
print(a)

b = np.einsum("ij->ji", a)
print("b = a.T")
print(b)

a = 
[[0 1 2]
 [3 4 5]]
b = a.T
[[0 3]
 [1 4]
 [2 5]]


### 2. Sum

In [3]:
# Sum of all matrix elements
c = np.einsum("ij->", a)
print("c = sum(a)")
print(c)

c = sum(a)
15


In [4]:
# Column sum
d = np.einsum("ij->j", a)
print("d = sum(a, axis=0)")
print(d)

# Row sum
e = np.einsum("ij->i", a)
print("e = sum(a, axis=1)")
print(e)

d = sum(a, axis=0)
[3 5 7]
e = sum(a, axis=1)
[ 3 12]


### 3. Matrix vector multiplication

In [5]:
b = np.arange(3)
print("b = ")
c = np.einsum("ij,j->i", a, b)
print("c = a*b")
print(c)

b = 
c = a*b
[ 5 14]


### 4. Matrix matrix multiplication

In [6]:
b = np.arange(15).reshape(3,5)
print("a = ")
print(a)
print("b = ")
print(b)
c = np.einsum("ik,kj->ik", a, b)
print("c = a*b")
print(c)

a = 
[[0 1 2]
 [3 4 5]]
b = 
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]
c = a*b
[[  0  35 120]
 [ 30 140 300]]


### 5. Dot product

In [7]:
# For vectors
a = np.arange(3)
b = np.arange(3, 6)
print("a = ")
print(a)
print("b = ")
print(b)

c = np.einsum("i,i->", a, b)
print("c = sum(a*b)")
print(c)

a = 
[0 1 2]
b = 
[3 4 5]
c = sum(a*b)
14


In [8]:
# For matrices
a = np.arange(6).reshape(2,3)
b = np.arange(6, 12).reshape(2,3)

print("a = ")
print(a)
print("b = ")
print(b)

c = np.einsum("ij,ij->", a, b)
print("c = sum(a*b)")
print(c)

a = 
[[0 1 2]
 [3 4 5]]
b = 
[[ 6  7  8]
 [ 9 10 11]]
c = sum(a*b)
145


### 6. Hadamard product

In [9]:
a = np.arange(6).reshape(2, 3)
b = np.arange(6, 12).reshape(2, 3)

print("a = ")
print(a)
print("b = ")
print(b)

c = np.einsum("ij,ij->ij", a, b)
print("Hadamard product c = a*b")
print(c)

a = 
[[0 1 2]
 [3 4 5]]
b = 
[[ 6  7  8]
 [ 9 10 11]]
Hadamard product c = a*b
[[ 0  7 16]
 [27 40 55]]


### 7. Outer product

In [10]:
a = np.arange(3)
b = np.arange(3, 7)

c = np.einsum("i,j->ij", a, b)
print("Outer product, c = a*b")
print(c)

Outer product, c = a*b
[[ 0  0  0  0]
 [ 3  4  5  6]
 [ 6  8 10 12]]


### 8. Batch matrix multiplication

In [11]:
a = np.random.rand(3, 2, 5)
b = np.random.rand(3, 5, 3)

print("a = ")
print(a)
print("b = ")
print(b)

c = np.einsum("Bjk,Bkl->Bjl", a, b)
print("Batch product, c = a*b")
print(c)

a = 
[[[0.11532823 0.16237904 0.48193012 0.53859684 0.74163052]
  [0.54384971 0.19226851 0.4792663  0.06543065 0.62855029]]

 [[0.11296697 0.67418912 0.16340693 0.77999147 0.27425265]
  [0.69752247 0.08261903 0.59917323 0.40249084 0.43276849]]

 [[0.21446412 0.94697481 0.83463772 0.58740656 0.47658791]
  [0.52880202 0.64109012 0.42169806 0.38004957 0.73772688]]]
b = 
[[[0.5529575  0.24301083 0.3412316 ]
  [0.31283246 0.29837214 0.9103908 ]
  [0.41782172 0.68005651 0.65206538]
  [0.21400499 0.26294805 0.4881443 ]
  [0.25098953 0.21114183 0.86519107]]

 [[0.44973135 0.4390371  0.58142883]
  [0.91823783 0.05038746 0.455185  ]
  [0.65625511 0.60175173 0.91553111]
  [0.41598256 0.67421109 0.01232127]
  [0.28374976 0.77049862 0.45042271]]

 [[0.08380452 0.31465664 0.63239868]
  [0.95941454 0.39901339 0.24369792]
  [0.72341856 0.9579315  0.85383178]
  [0.99711738 0.28874679 0.5498366 ]
  [0.99740897 0.13276061 0.07099713]]]
Batch product, c = a*b
[[[0.61733382 0.70242732 1.40599705]
  [0.7328

### 9. A more complex example

We will compute the independant vector equal to the sum over x, of the steepest descent image DIJ with the difference image DI, where x is a position in the image.

For the sake of simplicity of this example, we assume that images have only one (color) channel, and that the parameters registered in DIJ are those of a translation, ie. only two parameters per pixel.

In [12]:
dij = np.arange(72).reshape(4, 3, 3, 2)
di = np.arange(36).reshape(4, 3, 3)
dijt = np.einsum("ijlk->ijkl", dij)
print("dijt shape = ", dijt.shape)
prod = np.einsum("ijkl, ijl -> ijk", dijt, di)
print("prod shape = ", prod.shape)
b = np.einsum("ijl -> l", prod)
print("b shape = ", b.shape)
print(b)

dijt shape =  (4, 3, 2, 3)
prod shape =  (4, 3, 2)
b shape =  (2,)
[29820 30450]


In [13]:
# trying DIJ[i,j,:,:].T @ DI[i,j,:] 
prod = np.einsum("ijkl, ijlm -> km", dijt, dij)
print("prod shape = ", prod.shape)
print(prod)

prod shape =  (2, 2)
[[59640 60900]
 [60900 62196]]
